Definimos las funciones matematicas necesarias para poder llevar a cabo el protocolo RSA

In [17]:
import random
from math import gcd
import secrets


#codigo basado en https://coderoasis.com/implementing-rsa-from-scratch-in-python/
def _sqmatrixmul(m1, m2, w, mod):
	mr = [[0 for j in range(w)] for i in range(w)]
	for i in range(w):
		for j in range(w):
			for k in range(w):
				mr[i][j] =(mr[i][j]+m1[i][k]*m2[k][j])%mod
	return mr

# fibonacci calculator
def _fib(x, mod):
	if x < 3: return 1
	x -= 2
	# find length of e in bits
	tst = 1
	siz = 0
	while x >= tst:
		tst <<= 1
		siz += 1
	siz -= 1
	# calculate the matrix
	fm = [
		# function matrix
		[0, 1],
		[1, 1]
	]
	rm = [
		# result matrix
		# (identity)
		[1, 0],
		[0, 1]
	]
	for i in range(siz, -1, -1):
		rm = _sqmatrixmul(rm, rm, 2, mod)
		if (x >> i) & 1:
			rm = _sqmatrixmul(rm, fm, 2, mod)

	# second row of resulting vector is result
	return (rm[1][0] + rm[1][1]) % mod

def _modpow(b, e, n):
	# find length of e in bits
	tst = 1
	siz = 0
	while e >= tst:
		tst <<= 1
		siz += 1
	siz -= 1
	# calculate the result
	r = 1
	for i in range(siz, -1, -1):
		r = (r * r) % n
		if (e >> i) & 1: r = (r * b) % n
	return r

def _genprime(siz):
	while True:
		num = (1 << (siz - 1)) + secrets.randbits(siz - 1) - 10;
		num -= num % 10
		num += 3 # 3 (mod 10)
		# heuristic test
		if _modpow(2, num - 1, num) ==1 and _fib(num + 1, num) ==0:
			return num
		num += 5 # 7 (mod 10)
		# heuristic test
		if _modpow(2, num - 1, num) ==1 and _fib(num + 1, num) ==0:
			return num
def _modInverse(a, m):
    m0 = m
    y = 0
    x = 1
 
    if (m == 1):
        return 0
 
    while (a > 1):
 
        # q is quotient
        q = a // m
 
        t = m
 
        # m is remainder now, process
        # same as Euclid's algo
        m = a % m
        a = t
        t = y
 
        # Update x and y
        y = x - q * y
        x = t
 
    # Make x positive
    if (x < 0):
        x = x + m0
 
    return x
def _generate_keypair(p, q):
        if p == q:
            raise ValueError('p and q cannot be equal')
        #n = pq
        n = p * q

        #Phi is the totient of n
        phi = (p-1) * (q-1)

        #Choose an integer e such that e and phi(n) are coprime
        e = random.randrange(1, phi)

        #Use Euclid's Algorithm to verify that e and phi(n) are comprime
        g = gcd(e, phi)
        while g != 1:
            e = random.randrange(1, phi)
            g = gcd(e, phi)

        #Use Extended Euclid's Algorithm to generate the private key
        d = _modInverse(e, phi)
        #Return public and private keypair
        #Public key is (e, n) and private key is (d, n)
        return e, n, d


def _byte_length(i):
      return (i.bit_length() + 7) // 8






Implementacion de las clases que interactuan entre ellas siguiendo el protocolo de encriptacion por bloques

In [18]:
class RSAReceiver :
    def __init__ ( self , bit_len : int ) -> None :
      p = _genprime(bit_len)
      q = _genprime(bit_len)
      self.e, self.n, self.d = _generate_keypair(p,q)
      """
      Arguments :
      bit_len : A lower bound for the number of bits of N,
      the second argument of the public and secret key .
      """

    def get_public_key ( self ) -> bytearray :

      largo_bytes_e = _byte_length(self.e)
      largo_bytes_n = _byte_length(self.n)
      bloque_largo_e = largo_bytes_e.to_bytes(4, 'big')
      bloque_e = self.e.to_bytes(largo_bytes_e, 'big')
      bloque_largo_n = largo_bytes_n.to_bytes(4, 'big')
      bloque_n = self.n.to_bytes(largo_bytes_n, 'big')
      return bloque_largo_e + bloque_e + bloque_largo_n + bloque_n
    """
    Returns :
    public_key : Public key expressed as a Python ’bytearray ’ using the
    PEM format . This means the public key is divided in:
    (1) The number of bytes of e (4 bytes )
    (2) the number e (as many bytes as indicated in (1))
    (3) The number of bytes of N (4 bytes )
    (4) the number N (as many bytes as indicated in (3))
    """
    def decrypt ( self , ciphertext : bytearray ) -> str :
      largo_bloque = ((self.n.bit_length() - 1) // 8) + 1
      bloques = [ciphertext[i:i+largo_bloque] for i in range(0, len(ciphertext), largo_bloque)]
      dec = ""
      for indice in range(len(bloques)):
        m = int.from_bytes(bloques[indice], "big")
        #problema aca
        d =pow(m, self.d, self.n)
        d = d.to_bytes(largo_bloque - 1,byteorder="big")
        if indice == len(bloques)-1:
          d = bytearray(b for b in d if b != 0)
        dec += d.decode("utf-8") 
      return dec
  
    """
    Arguments :
    ciphertext : The ciphertext to decrypt
    Returns :
    message : The original message
    """

class RSASender :
    def __init__ ( self , public_key : bytearray ) -> None :
      largo_e = int.from_bytes( public_key[:4], 'big')
      self.e = int.from_bytes(public_key[4:4+largo_e], "big")
      largo_n = int.from_bytes( public_key[4+largo_e:8+largo_e], 'big')
      self.n = int.from_bytes(public_key[8 + largo_e: 8+ largo_e + largo_n], "big")
    """
    Arguments :
    public_key : The public key that will be used to encrypt messages
    """
    def encrypt ( self , message : str ) -> bytearray :
      array = bytearray(message, "utf-8")
      largo_bloque = (self.n.bit_length() - 1) // 8
      bloques = [array[i:i+largo_bloque] for i in range(0, len(array), largo_bloque)]
      enc = bytearray()
      for bloque in bloques:
        m = int.from_bytes(bloque, "big")
        c=pow(m,self.e, self.n)
        c = c.to_bytes(largo_bloque+1,byteorder="big")
        enc += c
      return enc
    """
    Arguments :
    message : The plaintext message to encrypt
    Returns :
    ciphertext : The encrypted message
    """